### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_finetune")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data = pd.read_csv('project.csv', index_col=0)
data

,summary,config,name
0,{'_wandb': {'runtime': 1}},"{'exp_name': 'byol_impala', 'overrides': [], '...",dry-terrain-2830
1,"{'_step': 100000, 'train_loss': 0.581707725083...","{'env': {'game': 'up_n_down', 'type': 'atari',...",woven-music-2829
2,"{'_runtime': 16183, '_timestamp': 1660476867, ...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",toasty-pond-2826
3,"{'_timestamp': 1660477092, 'train_mean_traj_re...","{'env': {'game': 'seaquest', 'type': 'atari', ...",sage-sun-2827
4,"{'_runtime': 16480, '_timestamp': 1660477164, ...","{'env': {'game': 'qbert', 'type': 'atari', 'ho...",dry-elevator-2828
...,...,...,...
2416,"{'train_mean_traj_rewards': 2.5, 'train_mean_t...","{'env': {'game': 'breakout', 'type': 'atari', ...",sandy-moon-11
2417,"{'_step': 1000, '_runtime': 14, '_timestamp': ...","{'env': {'game': 'alien', 'type': 'atari', 'ho...",twilight-darkness-16
2418,"{'_step': 1000, '_runtime': 13, '_timestamp': ...","{'env': {'game': 'qbert', 'type': 'atari', 'ho...",faithful-dust-4
2419,"{'_step': 1000, '_runtime': 14, '_timestamp': ...","{'env': {'game': 'breakout', 'type': 'atari', ...",smooth-smoke-13


### Filter based on conditions

In [3]:
configs = data['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    exp_name = cfg['exp_name']
    
    # condition
    if exp_name == 'byol_impala':
        indexs.append(idx)

In [4]:
data = data.iloc[indexs]
data

,summary,config,name
0,{'_wandb': {'runtime': 1}},"{'exp_name': 'byol_impala', 'overrides': [], '...",dry-terrain-2830
1,"{'_step': 100000, 'train_loss': 0.581707725083...","{'env': {'game': 'up_n_down', 'type': 'atari',...",woven-music-2829
2,"{'_runtime': 16183, '_timestamp': 1660476867, ...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",toasty-pond-2826
3,"{'_timestamp': 1660477092, 'train_mean_traj_re...","{'env': {'game': 'seaquest', 'type': 'atari', ...",sage-sun-2827
4,"{'_runtime': 16480, '_timestamp': 1660477164, ...","{'env': {'game': 'qbert', 'type': 'atari', 'ho...",dry-elevator-2828
...,...,...,...
2060,"{'eval_mean_traj_rewards': 53.900001525878906,...","{'env': {'game': 'alien', 'type': 'atari', 'ho...",fresh-spaceship-761
2061,"{'_step': 100000, '_timestamp': 1659717638, 'e...","{'env': {'game': 'boxing', 'type': 'atari', 'h...",floral-snow-766
2062,"{'_wandb': {'runtime': 28053}, '_timestamp': 1...","{'env': {'game': 'crazy_climber', 'type': 'ata...",clean-firebrand-759
2063,"{'_wandb': {'runtime': 27831}, '_timestamp': 1...","{'env': {'game': 'battle_zone', 'type': 'atari...",glowing-armadillo-764


### Generate json file

In [17]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)